In [3]:
import pandas as pd
import datetime
import numpy as np
from private_stuff.nombres import normalizacion_nombres
from private_stuff.nacionalidades import normalizacion_paises
from private_stuff.institucion import normalizacion_universidades
from funciones import limpiar_caracteres_especiales, limpiar_funciones, instrumento_financiamiento, normalizacion_ciudad, ciudad_a_region
from funciones import institucion_instrumento, descripcion_codigo_inach, region_institucion, macrozona_region, proyecto_concursable
from area_ocde import equivalencia_objetivos

In [4]:
lista_hojas = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
xls_base = pd.read_excel(f"raw-data/procien/PROCIEN-proyectos.xlsx", sheet_name=lista_hojas)


In [5]:
xls_procien_antartic = pd.read_excel(f"raw-data/procien/PROCIEN-ANID.xlsx")[['codigo-procien', 'codigo-anid']]
dict_procien_anid = {}
for row in xls_procien_antartic.to_numpy().tolist():
    if str(row[1]).strip() not in ['noaplica', 'noinf', 's/i', 'corfo', 'proyecto internacional']:
        dict_procien_anid[ row[0] ] = str(row[1])

xls_dinero_inach = pd.read_excel( f"raw-data/procien/PROCIEN-presupuestos.xlsx", sheet_name=['pagina inach'] )
xls_dinero_inach = xls_dinero_inach['pagina inach'][['codigo', 'monto-total']]
dict_procien_inach = {}
for row in xls_dinero_inach.to_numpy().tolist():
    dict_procien_inach[ row[0].strip().replace('_', '-') ] = float(row[1])

xls_anid = pd.read_excel(f"anid/ANID-antartic.xlsx")[['CODIGO_PROYECTO', 'MONTO_ADJUDICADO']]

In [6]:
def buscar_monto_anid( codigo_anid, xls_anid ):
    for row in xls_anid.to_numpy().tolist():
        if str(row[0])==codigo_anid:
            return float( row[1] )*1000.0
    return 0.0

### Eliminar personas duplicadas
Lo que se hizo fue tomar cada hoja del procien y eliminar las personas duplicadas bajo el criterio que compartan mismo codigo y mismo anio procien.
En caso de que uno de estos dos sea diferente, esa persona se mantiene.

In [ ]:
new_xls = []
lista_unicos = []

for n in lista_hojas:
    tmp_xls = xls_base[n][['codigo', 'titulo', 'pi', 'ciudad', 'persona', 'inst', 'pais', 'genero']]
    
    for row in tmp_xls.to_numpy().tolist():
        codigo, titulo, pi, ciudad, persona, inst, pais, genero = row

        codigo = codigo.split( ' ' )[0].replace( '_', '-' ).upper()
        concursable =  proyecto_concursable( codigo.split('-')[0] )
        proyecto, tesis, terreno = descripcion_codigo_inach( codigo )
        instrumento = instrumento_financiamiento( codigo.split('-')[0] )
        inst_instrumento = institucion_instrumento( codigo )

        titulo = limpiar_caracteres_especiales( titulo.lower() )

        inst = limpiar_caracteres_especiales( inst.lower().strip() )
        inst = normalizacion_universidades[ inst ]
        region_inst = region_institucion[inst]
        macro_inst = macrozona_region[region_inst]

        pais = normalizacion_paises[ limpiar_caracteres_especiales( pais.lower() ).strip().upper() ].lower().strip()
        persona = normalizacion_nombres[ limpiar_caracteres_especiales( persona.lower().strip() ) ]

        ciudad = normalizacion_ciudad[ limpiar_caracteres_especiales( ciudad.lower().strip() ) ]
        region_ciudad = ciudad_a_region[ciudad]
        macro_ciudad = macrozona_region[region_ciudad]

        pi = limpiar_caracteres_especiales( pi.lower() ).strip()
        if '/' in pi:
            pi = pi.split('/')[0].strip()
        elif '-' in pi:
            pi = pi.split('-')[0].strip()
        pi = normalizacion_nombres[pi]

        es_pi = 'asociado'
        if persona==pi:
            es_pi = 'pi'

        ##se busca que las personas no aparescan mas de una vez por anio y por codigo
        if [persona, n, codigo] not in lista_unicos:
            new_xls.append([n, codigo, proyecto, tesis, terreno, concursable, instrumento, inst_instrumento, titulo, pi, 
                inst, region_inst, macro_inst, pais, persona, ciudad, region_ciudad, macro_ciudad, genero, es_pi]) 
            lista_unicos.append( [persona, n, codigo] )
        
    print(f"{n} done")

In [6]:
full_dataframe = pd.DataFrame(np.array(new_xls), columns=['anio', 'codigo', 'tipo-proyecto', 'tesis', 
    'terreno', 'consursable', 'instrumento', 'inst-instrumento', 'titulo', 'pi', 'inst', 'region-inst', 'macro-inst',
    'pais', 'persona', 'ciudad-pi', 'region-pi', 'macro-pi', 'genero', 'es-pi'])
full_dataframe.to_excel(f"procien/PROCIEN-just-personas.xlsx")

### Codigos por anio
Lo que se hace es dejar los codigos unicos que aparecen en un anio, si el mismo codigo aparece en otro anio, este se mantiene.

In [ ]:
new_xls = []
list_codes_year = []

for n in lista_hojas:
    tmp_xls = xls_base[n][['codigo', 'titulo', 'pi', 'ciudad', 'persona', 'inst', 'pais', 'genero']]
    
    for row in tmp_xls.to_numpy().tolist():
        codigo, titulo, pi, ciudad, persona, inst, pais, genero = row

        codigo = codigo.split( ' ' )[0].replace( '_', '-' ).upper()
        concursable =  proyecto_concursable( codigo.split('-')[0] )
        o1, o2, o3 = equivalencia_objetivos[codigo]
        proyecto, tesis, terreno = descripcion_codigo_inach( codigo )
        instrumento = instrumento_financiamiento( codigo.split('-')[0] )
        inst_instrumento = institucion_instrumento( codigo )

        titulo = limpiar_caracteres_especiales( titulo.lower() )

        ciudad = normalizacion_ciudad[ limpiar_caracteres_especiales( ciudad.lower().strip() ) ]
        region_ciudad = ciudad_a_region[ciudad]
        macro_ciudad = macrozona_region[region_ciudad]

        pi = limpiar_caracteres_especiales( pi.lower() ).strip()
        if '/' in pi:
            pi = pi.split('/')[0].strip()
        elif '-' in pi:
            pi = pi.split('-')[0].strip()
        pi = normalizacion_nombres[pi]

        if [codigo, n] not in list_codes_year:
            new_xls.append([n, codigo, proyecto, tesis, terreno, concursable, instrumento, inst_instrumento, titulo, pi, 
                    ciudad, region_ciudad, macro_ciudad, o1, o2, o3]) 
            list_codes_year.append( [codigo, n] )
        
    print(f"{n} done")

In [8]:
full_dataframe = pd.DataFrame(np.array(new_xls), columns=['anio', 'codigo', 'tipo-proyecto', 'tesis', 
    'terreno', 'consursable', 'instrumento', 'inst-instrumento', 'titulo', 'pi', 'ciudad-pi', 'region-pi', 
    'macro-pi', 'ocde', 'inach', 'anid'])
full_dataframe.to_excel(f"procien/PROCIEN-just-codes-by-year.xlsx")

### Just Unique codes

In [ ]:
new_xls = []
list_codes = []

for n in lista_hojas:
    tmp_xls = xls_base[n][['codigo', 'titulo', 'pi', 'ciudad', 'persona', 'inst', 'pais', 'genero']]
    
    for row in tmp_xls.to_numpy().tolist():
        codigo, titulo, pi, ciudad, persona, inst, pais, genero = row

        codigo = codigo.split( ' ' )[0].replace( '_', '-' ).upper()
        concursable =  proyecto_concursable( codigo.split('-')[0] )
        o1, o2, o3 = equivalencia_objetivos[codigo]
        proyecto, tesis, terreno = descripcion_codigo_inach( codigo )
        instrumento = instrumento_financiamiento( codigo.split('-')[0] )
        inst_instrumento = institucion_instrumento( codigo )

        titulo = limpiar_caracteres_especiales( titulo.lower() )


        ciudad = normalizacion_ciudad[ limpiar_caracteres_especiales( ciudad.lower().strip() ) ]
        region_ciudad = ciudad_a_region[ciudad]
        macro_ciudad = macrozona_region[region_ciudad]
        inst = normalizacion_universidades[ limpiar_caracteres_especiales( inst.lower().strip() ) ]

        pi = limpiar_caracteres_especiales( pi.lower() ).strip()
        if '/' in pi:
            pi = pi.split('/')[0].strip()
        elif '-' in pi:
            pi = pi.split('-')[0].strip()
        pi = normalizacion_nombres[pi]

        codigo_anid = 'noaplica'
        if codigo in dict_procien_anid.keys():
            codigo_anid = dict_procien_anid[ codigo ]

        monto=0
        if codigo_anid != 'noaplica':
            monto = buscar_monto_anid( codigo_anid, xls_anid )
        elif inst_instrumento == 'INACH':
            if codigo in dict_procien_inach.keys(): 
                monto = dict_procien_inach[codigo]
            else:
                monto = 0

        code_anio = f"20{codigo.split('-')[2]}"
        persona = normalizacion_nombres[ limpiar_caracteres_especiales( persona.lower().strip() ) ]

        if codigo not in list_codes:
            ##se agrega la informacion del pi
            if pi == persona:
                new_xls.append([code_anio, codigo, proyecto, tesis, terreno, concursable, instrumento, inst_instrumento, titulo, pi, 
                    ciudad, region_ciudad, macro_ciudad, o1, o2, o3, codigo_anid, int(monto), inst, genero, n]) 
                list_codes.append( codigo )
        
    print(f"{n} done")

In [10]:
full_dataframe = pd.DataFrame(np.array(new_xls), columns=['anio', 'codigo', 'tipo-proyecto', 'tesis', 
    'terreno', 'consursable', 'instrumento', 'inst-instrumento', 'titulo', 'pi', 'ciudad-pi', 'region-pi', 
    'macro-pi', 'ocde', 'inach', 'anid', 'codigo-anid', 'monto', 'inst-pi', 'genero-pi', 'anio-procien'])
full_dataframe.to_excel(f"procien/PROCIEN-just-codes.xlsx")

### Unificacion de ECAs y PROCIEN

In [25]:
lista_hojas = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
lista_archivos = ['50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60']

eca_anio = {
    '50': '2014',
    '51': '2015',
    '52': '2016',
    '53': '2017',
    '54': '2018',
    '55': '2019',
    '56': '2020',
    '57': '2021',
    '58': '2022',
    '59': '2023',
    '60': '2024',
}

anio_eca = {
    '2014': ['50', 0],
    '2015': ['51', 1],
    '2016': ['52', 2],
    '2017': ['53', 3],
    '2018': ['54', 4],
    '2019': ['55', 5],
    '2020': ['56', 6],
    '2021': ['57', 7],
    '2022': ['58', 8],
    '2023': ['59', 9],
    '2024': ['60', 10],
}


In [26]:
xls_base_procien = pd.read_excel(f"raw-data/procien/PROCIEN-proyectos.xlsx", sheet_name=lista_hojas)
xls_list = []
for f in lista_archivos:
    if f in ['50', '51', '55', '56', '57']:
        xls_list.append( pd.read_excel(f"raw-data/eca/ECA {f}.xlsx") )
    else:
        xls_list.append( pd.read_excel(f"raw-data/eca/ECA {f}.xlsx", sheet_name=['personas'])['personas'] )

#### Sacar los codigos junto anio eca

In [ ]:
list_codigos = []
suma = 0
for i,xls in enumerate(xls_list):
    for row in xls['codigo'].to_numpy().tolist():
        codigo = row
        codigo = codigo.replace('_', '-').split(' ')[0]
        if codigo not in list_codigos:
            list_codigos.append( codigo )

for anio in lista_hojas:
    tmp_xls = xls_base_procien[anio]['codigo'].to_numpy().tolist()
    for row in tmp_xls:
        codigo = row
        codigo = codigo.replace('_', '-').split(' ')[0]
        suma += 1
        if codigo not in list_codigos:
            list_codigos.append( codigo )
print(suma)


#### Funciones para extraer anios y grupos de personas

In [28]:
def identificar_apariciones_procien( codigo, xls ):
    lista_apariciones = []
    for hoja in lista_hojas:
        lista = [ c.replace('_', '-').split(' ')[0] for c in xls[hoja]['codigo'].to_numpy().tolist() ]
        if codigo in lista and hoja not in lista_apariciones:
            lista_apariciones.append(hoja)
    return lista_apariciones


def identificar_apariciones_eca( codigo, lista_xls ):
    lista_apariciones = []
    for i,xls in enumerate( lista_xls ):
        lista = [ c.replace('_', '-').split(' ')[0] for c in xls['codigo'].to_numpy().tolist() ]
        if codigo in lista and i not in lista_apariciones:
            lista_apariciones.append( i )
    return lista_apariciones

def lista_personas_procien(codigo, anio, xls):
    lista_personas = []
    tmp_xls = xls[anio][['codigo', 'persona']].to_numpy().tolist()
    for row in tmp_xls:
        if codigo==row[0].replace('_', '-').split(' ')[0]: lista_personas.append( row )
    return lista_personas


def lista_personas_eca(codigo, xls):
    lista_personas = []
    tmp_xls = xls[['codigo', 'nombre', 'apellido']].to_numpy().tolist()
    for row in tmp_xls:
        if codigo==row[0].replace('_', '-').split(' ')[0]: 
            lista_personas.append( [codigo, f'{row[1].strip()} {row[2].strip()}'] )
    return lista_personas

def buscar_personas_procien( persona, codigo, anio, xls ):
    tmp_xls = xls[anio][['codigo', 'persona']].to_numpy().tolist()
    for row in tmp_xls:
        persona_row = normalizacion_nombres[ limpiar_caracteres_especiales( row[1].lower().strip() ) ]
        if codigo==row[0].replace('_', '-').split(' ')[0] and persona == persona_row:
            return 'si'
    return 'no'

def buscar_personas_eca( persona, codigo, xls):
    tmp_xls = xls[['codigo', 'nombre', 'apellido']].to_numpy().tolist()
    for row in tmp_xls:
        persona_row = f'{row[1].strip()} {row[2].strip()}'.lower()
        persona_row = normalizacion_nombres[ limpiar_caracteres_especiales( persona_row ) ]
        if codigo==row[0].replace('_', '-').split(' ')[0] and persona == persona_row:
            return 'si'
    return 'no'

def rol_persona_eca( persona, codigo, xls):
    tmp_xls = xls[['codigo', 'nombre', 'apellido', 'funcion']].to_numpy().tolist()
    for row in tmp_xls:
        persona_row = f'{row[1].strip()} {row[2].strip()}'.lower()
        persona_row = normalizacion_nombres[ limpiar_caracteres_especiales( persona_row ) ]
        if codigo==row[0].replace('_', '-').split(' ')[0] and persona == persona_row:
            return limpiar_funciones( limpiar_caracteres_especiales( row[3].lower() ) )
    return 'noinf'

def es_pi( persona, codigo, anio, xls ):
    tmp_xls = xls[anio][['codigo', 'pi']].to_numpy().tolist()
    for row in tmp_xls:
        persona_row = limpiar_caracteres_especiales( row[1].lower().strip() )
        if '/' in persona_row:
            persona_row = persona_row.split('/')[0].strip()
        elif '-' in persona_row:
            persona_row = persona_row.split('-')[0].strip()
            
        persona_row = normalizacion_nombres[ persona_row ]
        if codigo==row[0].replace('_', '-').split(' ')[0] and persona == persona_row:
            return True
    return False

In [29]:
new_xls = []
lista_repetidos = []

for codigo in list_codigos:
    lista_prociens = identificar_apariciones_procien( codigo, xls_base_procien )
    lista_ecas = identificar_apariciones_eca( codigo, xls_list )

    ##Determinar los anios
    lista_anios = []
    for anio in lista_prociens:
        if anio not in lista_anios: lista_anios.append( anio )
    
    for num_eca in lista_ecas:
        if eca_anio[ lista_archivos[num_eca] ] not in lista_anios: lista_anios.append( eca_anio[ lista_archivos[num_eca] ] )

    ##Agrupar personas por anio
    lista_personas = []
    for anio in lista_prociens:
        lista_procien = lista_personas_procien( codigo, anio, xls_base_procien )
        for row in lista_procien:
            lista_personas.append( [anio, row[1]] )    
    for num_eca in lista_ecas:
        lista_eca = lista_personas_eca( codigo, xls_list[num_eca] )
        for row in lista_eca:
            lista_personas.append( [eca_anio[ lista_archivos[num_eca] ], row[1]] )
        
    ##Normalizar nombres de personas por anio
    for year, person in lista_personas:
        eca,idx = anio_eca[ year ] 
        person = normalizacion_nombres[ limpiar_caracteres_especiales( person.lower().strip() ) ]
                
        esta_en_procien = buscar_personas_procien( person, codigo, year, xls_base_procien )
        esta_en_eca = buscar_personas_eca( person, codigo, xls_list[idx] )

        #determinar_rol
        if esta_en_procien=='si':
            rol = 'Investigador'
            if es_pi( person, codigo, year, xls_base_procien ): rol = 'Investigador Principal'
        elif eca in ['51', '53', '57', '54']:
            rol = 'noinf'
        else:
            rol = rol_persona_eca( person, codigo, xls_list[idx] )

        new_row = [ codigo, year, person, esta_en_eca, esta_en_procien, rol]
        if new_row not in new_xls: 
            new_xls.append( new_row )

In [30]:
full_dataframe = pd.DataFrame(np.array(new_xls), columns=['codigo', 'anio', 
        'persona', 'esta-en-eca', 'esta-en-procien', 'rol'])
full_dataframe.to_excel(f"procien/PROCIEN-ECA-personas.xlsx")

In [31]:
xls_eca_procien = new_xls.copy()

dict_personas_procien = {
    '2014': [], '2015': [], '2016': [], '2017': [],
    '2018': [], '2019': [], '2020': [], '2021': [],
    '2022': [], '2023': [], '2024': [],
}


dict_personas_eca = {
    '2014': [], '2015': [], '2016': [], '2017': [],
    '2018': [], '2019': [], '2020': [], '2021': [],
    '2022': [], '2023': [], '2024': [],
}


dict_personas_global = {
    '2014': [], '2015': [], '2016': [], '2017': [],
    '2018': [], '2019': [], '2020': [], '2021': [],
    '2022': [], '2023': [], '2024': [],
}


for row in xls_eca_procien:
    codigo, anio, persona, esta_en_eca, esta_en_procien, _ = row 
    if persona not in dict_personas_global[anio]:
        dict_personas_global[anio].append(persona)

    if persona not in dict_personas_procien[anio] and esta_en_procien == 'si':
    #if esta_en_procien == 'si':
      dict_personas_procien[anio].append(persona)

    if persona not in dict_personas_eca[anio] and esta_en_eca == 'si':
    #if esta_en_eca == 'si':
      dict_personas_eca[anio].append(persona)

In [32]:
def personas_no_repedidas_global(dict_personas_anio):
    pool_nombres = dict_personas_anio['2014'].copy()
    lista_nuevos = [ len( dict_personas_anio['2014'] ) ]
    lista_universo = [ len( dict_personas_anio['2014'] ) ]

    for anio in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']:
        numero = 0
        for p in dict_personas_anio[anio]:
            if p not in pool_nombres:
                pool_nombres.append( p )
                numero = numero + 1
        lista_universo.append( len( dict_personas_anio[anio] ) )
        lista_nuevos.append( numero )
    return lista_nuevos, lista_universo

def personas_no_repedidas_consecutivo(dict_personas_anio):
    pool_nombres = dict_personas_anio['2014'].copy()
    lista_nuevos = [ len( dict_personas_anio['2014'] ) ]
    lista_universo = [ len( dict_personas_anio['2014'] ) ]
    lista_anios = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

    for i,anio in enumerate(lista_anios):
        if anio != '2015':
            pool_nombres = dict_personas_anio[lista_anios[i-1]]
        numero = 0
        for p in dict_personas_anio[anio]:
            if p not in pool_nombres:
                numero = numero + 1
        lista_universo.append( len( dict_personas_anio[anio] ) )
        lista_nuevos.append( numero )
    return lista_nuevos, lista_universo


In [ ]:
p1_global, uni_global = personas_no_repedidas_consecutivo(dict_personas_global)
p2_global, uni_global = personas_no_repedidas_global(dict_personas_global)
print('pool consecutivo: ', p1_global)
print('pool global: ', p2_global)
print('univ personas: ', uni_global)

In [ ]:
p1_procien, uni_procien = personas_no_repedidas_consecutivo(dict_personas_procien)
p2_procien, uni_procien = personas_no_repedidas_global(dict_personas_procien)
print('pool consecutivo: ', p1_procien)
print('pool global: ', p2_procien)
print('univ personas: ', uni_procien)

In [ ]:
p1_eca, uni_eca = personas_no_repedidas_consecutivo(dict_personas_eca)
p2_eca, uni_eca = personas_no_repedidas_global(dict_personas_eca)
print('pool consecutivo: ', p1_eca)
print('pool global: ', p2_eca)
print('univ personas: ', uni_eca)

In [22]:
lista_anios = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
new_xls = np.array([lista_anios, p1_eca, p2_eca, uni_eca, p1_procien, p2_procien, uni_procien, p1_global, p2_global, uni_global])

In [24]:
fecha = datetime.datetime.now()
new_xls = pd.DataFrame( new_xls.T, columns=['anio', 'nuevas-global-eca', 'nuevas-consecutivo-eca', 'universo-eca',
        'nuevas-global-procien', 'nuevas-consecutivo-procien', 'universo-procien', 
        'nuevas-global-global', 'nuevas-consecutivo-global', 'universo-global'])
new_xls.to_excel(f"procien/PROCIEN-ECA-detalle-repetidos.xlsx")